<a href="https://colab.research.google.com/github/Milanasa/tutor-quests/blob/main/testSQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime,date
import sqlite3

In [ ]:
# Импортируем таблицу с данными о погоде за 14 лет
wr = pd.read_csv('/content/drive/MyDrive/weatherHARD.csv', sep = ';', encoding = 'UTF-8', skiprows = 6, low_memory=False)
df = pd.DataFrame(wr)
display(df)

In [ ]:
df_main = df['Местное время в Туле']

In [ ]:
DF = pd.DataFrame(df_main)
display(DF)
DF_COPY = DF

In [ ]:
NEW_DF = DF_COPY.reset_index()
display(NEW_DF)

In [ ]:
DF_WITH_DATE = NEW_DF['index'].str.split(' ', expand=True)
DF_WITH_DATE = DF_WITH_DATE.rename(columns = {0: 'Дата'})
DF_WITH_DATE = DF_WITH_DATE.rename(columns = {1: 'Время'})

display(DF_WITH_DATE)

In [ ]:
DF_WITH_DATE = DF_WITH_DATE.join(NEW_DF['Местное время в Туле'])
DF_WITH_DATE = DF_WITH_DATE.rename(columns = {'Местное время в Туле': 'Температура'})
display(DF_WITH_DATE)

In [ ]:
DATE = DF_WITH_DATE['Дата']
TIME = DF_WITH_DATE['Время']
TEMP = DF_WITH_DATE['Температура']
DFFF_COPY = df.reset_index()
PRESSURE = DFFF_COPY['Po']

In [ ]:
# Запускаем базу данных
connection = sqlite3.connect('WEATHER_DATABASE.db')
cursor = connection.cursor()

In [ ]:
# Создаём SQLite таблицу
cursor.execute('''
CREATE TABLE IF NOT EXISTS Weather (
id INTEGER PRIMARY KEY,
date TEXT NOT NULL,
time TEXT NOT NULL,
temp REAL,
pressure REAL
)
''')

In [ ]:
# Ввод данных в таблицу
for i in range(len(DF_WITH_DATE.index)):
  cursor.execute('INSERT INTO Weather (date, time, temp, pressure) VALUES (?, ?, ?, ?)', (DATE.loc[i], TIME.loc[i], TEMP.loc[i], PRESSURE.loc[i]))

In [ ]:
cursor.execute('CREATE INDEX idx_date ON Weather (date)')
cursor.execute('CREATE INDEX idx_time ON Weather (time)')
cursor.execute('CREATE INDEX idx_temp ON Weather (temp)')

In [ ]:
# Находим дни, в которые температура была от -6 до -5 градусов цельсия
cursor.execute('SELECT date, temp FROM Weather WHERE temp <= -5 AND temp >= -6')

In [ ]:
results = cursor.fetchall()

In [ ]:
for row in results:
  print(row)

In [ ]:
# Создаём функцию для создания регулярных выражений
def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

# Подключаем созданную функцию в SQLite
connection.create_function("REGEXP", 2, regexp)

# С помощью регулярных выражений выводим дни, температуру и время 12 февраля за все года
cursor.execute('SELECT date, temp, time FROM Weather WHERE date REGEXP ?', [r'12.02'])
results = cursor.fetchall()

for row in results:
  print(row)

In [ ]:
# С помощью регулярных выражений ищем количество дней, когда температура была строго выше 0 градусов цельсия в 2016 году
cursor.execute('SELECT COUNT(date) FROM Weather WHERE date REGEXP ? AND temp > 0', [r'\d\d.\d\d.2016'])
results = cursor.fetchall()

print(results[0][0] / 9)

In [ ]:
# Выводим первые 5 дат, когда температура была -25 градусов цельсия
cursor.execute('SELECT date, temp FROM Weather WHERE temp = -25')
results = cursor.fetchmany(5)

for row in results:
  print(row)

# В результате всего 3 строки, так как всего 3 дня, когда температура была -25 градусов.
# Это можно проверить, уменьшив нужную температуру на один градус, будет 5 строк.

In [ ]:
# Ищем минимальную и максимальную температуру за 14 лет
cursor.execute('SELECT date, temp FROM Weather WHERE temp = (SELECT MAX(temp) FROM Weather)')
results = cursor.fetchone()

cursor.execute('SELECT date, temp FROM Weather WHERE temp = (SELECT MIN(temp) FROM Weather)')
results1 = cursor.fetchone()

print(f'Максимальная: {results}, минимальная {results1}')

In [ ]:
MOST_USEFUL_SQL_FUNCTIONS = {0 : 'SELECT', 1 : 'CREATE', 2 : ['MAX', 'MIN', 'AVG'], 3: ['FROM', 'WHERE', 'INTO']}